In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os, pickle, gc, random
from tensorflow.python.keras import layers, Sequential,losses, metrics, models
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.optimizer_v2 import adam

In [ ]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [ ]:
image_path = "./dataset/aligned_images.npy"
emotion_paths = ["./dataset/emotions_1.npy",
                "./dataset/emotions_2.npy",
                "./dataset/emotions_3.npy",
                "./dataset/emotions_4.npy",
                "./dataset/emotions_5.npy",
                "./dataset/emotions_6.npy",
                "./dataset/emotions_7.npy",
                "./dataset/emotions_multi.npy"]

images = np.load(image_path)
images = images/255.0

emotions1 = np.load(emotion_paths[0])
emotions2 = np.load(emotion_paths[1])
emotions3 = np.load(emotion_paths[2])
emotions4 = np.load(emotion_paths[3])
emotions5 = np.load(emotion_paths[4])
emotions6 = np.load(emotion_paths[5])
emotions7 = np.load(emotion_paths[6])
emotions8 = np.load(emotion_paths[7])
emotions_list = [emotions1, emotions2, emotions3, emotions4, emotions5, emotions6, emotions7, emotions8]

sample_size = emotions8.shape[0]
training_size = 28317 + 3541

test_subset_indices = []
for i in range(sample_size):
    if np.count_nonzero(emotions8[i] > 0) >= 4:
        test_subset_indices.append(i)
test_subset_indices = random.sample(test_subset_indices, k=sample_size-training_size)
train_subset_indices = [item for item in range(sample_size) if not item in test_subset_indices]

In [ ]:
training_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(images[train_subset_indices]))
test_images = tf.image.grayscale_to_rgb(tf.convert_to_tensor(images[test_subset_indices]))

print("training images shape:", training_images.shape)
print("test images shape:", test_images.shape)

In [ ]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [ ]:
def train(model, learning_rate, loss, num_epochs, batch_size, label):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    emotions = emotions_list[label-1]
    training_emotions = tf.convert_to_tensor(emotions[train_subset_indices])
    test_emotions = tf.convert_to_tensor(emotions[test_subset_indices])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [ ]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()
history_save_paths = ['./history/complex-emo1.txt',
                     './history/complex-emo2.txt',
                     './history/complex-emo3.txt',
                     './history/complex-emo4.txt',
                     './history/complex-emo5.txt',
                     './history/complex-emo6.txt',
                     './history/complex-emo7.txt',
                     './history/complex-emo8.txt',]
for label in range(8,0,-1):
    base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))
    model = create_model(base_model)
    history_save_path = history_save_paths[label-1]
    history = train(model, learning_rate, loss, num_epochs, batch_size, label)
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)